In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
data = pd.read_csv('../input/breast-cancer-wisconsin-data/data.csv')

data['perimeter_radius_Crs']=data['perimeter_worst']*data['radius_worst']


data.head(10)

In [ ]:
#covert string to binary
data.dropna(axis=1,inplace=True)
data.drop(columns='id',axis=1,inplace=True)

y=(data['diagnosis']=='M')

In [ ]:



feature_names=[i for i in data.columns if data[i].dtype in [np.int64,np.float64,np.bool_]]

X=data[feature_names]


X.head()


In [ ]:
train_X,val_X,train_y,val_y=train_test_split(X,y,random_state=1)
my_model=RandomForestClassifier(n_estimators=500, random_state=0,bootstrap=False).fit(train_X,train_y)
model2=GradientBoostingClassifier(n_estimators=1000,learning_rate=0.05,max_depth=3).fit(train_X,train_y)



# Make predictions for the test set
#y_pred = my_model.predict(val_X)

#predictions with GBC
#y_pred=model2.predict(val_X)
y_pred=my_model.predict(val_X)


# View accuracy score
acc=accuracy_score(val_y, y_pred)
print(acc)



# View the classification report for test data and predictions
print(classification_report(val_y, y_pred))


# View confusion matrix for test data and predictions
confusion_matrix(val_y, y_pred)

In [ ]:
score=my_model.score(val_X,val_y)
print('Score:',score)
predictions=my_model.predict(val_X)
print('\n\n Predictions:',predictions)

In [ ]:
#Patinet wise prediction of breast cancer
patient_row=15
data_for_prediction=val_X.iloc[patient_row]
data_for_prediction_array=data_for_prediction.values.reshape(1,-1)
my_model.predict_proba(data_for_prediction_array)

res=my_model.predict(data_for_prediction_array)
print('Diaognosed with breast Cancer:',res)

In [ ]:
#compute SHAP
import shap
explainer=shap.TreeExplainer(my_model)
shap_values=explainer.shap_values(data_for_prediction)

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[1],shap_values[1],data_for_prediction)

In [ ]:
#compute shap values for all val_X
shap_values=explainer.shap_values(val_X)

In [ ]:
#make a plot
shap.summary_plot(shap_values[1],val_X)

In [ ]:
explainer=shap.TreeExplainer(my_model)
shap_values=explainer.shap_values(X)

shap.dependence_plot('perimeter_worst',shap_values[1],X)

In [ ]:
explainer=shap.TreeExplainer(my_model)
shap_values=explainer.shap_values(X)

shap.dependence_plot('perimeter_worst',shap_values[1],X,interaction_index='area_worst')

In [ ]:
explainer=shap.TreeExplainer(my_model)
shap_values=explainer.shap_values(X)

shap.dependence_plot('area_worst',shap_values[1],X,interaction_index='radius_worst')